Hi everyone,

Attached you have a dropbox link with the option data. A few things to note:
- You only have end of month date
- Option returns are delta hedged ONLY ONCE not on a daily basis. I went for this approach as it was easier to code so you will have the data faster.
- Only options which have prices at the next end of month are considered, again I did this because it was faster to code but it will probably not inhibit your results too much. 
- I have pre cleaned the option data for you. See for example Goyal (2009) for more details

For now the data seems super small (700mb) so I send you the whole data.

Best,
Patrick

In [1]:
# enable auto complete
%config Completer.use_jedi = False

In [2]:
from collections import *

In [3]:
import pandas as pd 

In [4]:
import datetime

In [5]:
pd.set_option('display.max_columns', 100)

# Data Exploration

In [7]:
sp500_op_ret = pd.read_csv("data/sp500_op_ret.csv")

In [8]:
mapping_table = pd.read_csv("data/mapping_table.csv")

In [9]:
sp500_const_list = pd.read_csv("data/sp500_const_list.csv")

## mapping_table and sp500_const_list

In [10]:
# PERMNO is a unique stock (share class) level 
# identifier used by WRDS
mapping_table.sample(5)

,secid,sdate,edate,permno
22885,205127,2014-08-27 00:00:00,2020-12-31 00:00:00,14845
13746,113188,2001-11-20 00:00:00,2012-02-22 00:00:00,89208
22228,190028,2014-01-09 00:00:00,2020-12-31 00:00:00,14412
18832,140393,2010-01-14 00:00:00,2020-12-31 00:00:00,93221
21404,161471,2013-02-15 00:00:00,2020-12-31 00:00:00,13779


In [12]:
mapping_table.shape

(28276, 4)

In [14]:
len(set(mapping_table['permno']))

22368

In [19]:
mapping_table[mapping_table["secid"]==5007]

,secid,sdate,edate,permno
5,5007,2001-04-09 00:00:00,2002-08-01 00:00:00,19692
6,5007,2012-04-12 00:00:00,2016-08-17 00:00:00,13343


In [18]:
Counter(mapping_table["secid"])

Counter({5001: 1,
         5002: 1,
         5004: 1,
         5005: 1,
         5006: 1,
         5007: 2,
         5008: 1,
         5009: 1,
         5010: 1,
         5012: 1,
         5014: 1,
         5015: 1,
         5016: 1,
         5017: 1,
         5019: 1,
         5021: 1,
         5022: 1,
         5023: 1,
         5024: 2,
         5025: 1,
         5027: 1,
         5028: 1,
         5029: 1,
         5030: 1,
         5031: 1,
         5033: 1,
         5034: 1,
         5035: 1,
         5036: 1,
         5037: 1,
         5038: 1,
         5040: 1,
         5041: 1,
         5042: 1,
         5043: 1,
         5044: 1,
         5045: 1,
         5046: 1,
         5047: 2,
         5048: 1,
         5049: 1,
         5051: 1,
         5052: 1,
         5053: 1,
         5056: 1,
         5057: 1,
         5058: 1,
         5059: 1,
         5060: 1,
         5061: 1,
         5064: 1,
         5065: 1,
         5066: 1,
         5067: 1,
         5068: 1,
         5

In [104]:
secids = set(sp500_op_ret.secid)
len(
    set(
        mapping_table[mapping_table.secid.apply(lambda x : x in secids)].permno
    )
)

1247

In [107]:
pd.DataFrame(set(
        mapping_table[mapping_table.secid.apply(lambda x : x in secids)].permno
    )).to_csv("permno_list.csv")

In [100]:
mapping_table.permno.unique().size

22368

In [24]:
sp500_const_list.sample(5)

,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending
1217,12570,INTERNATIONAL TEL & TELEG CORP,46047010,11,1,3561,IIN,5860,01,1957-03-01 00:00:00,2011-10-31 00:00:00
1852,22613,PUBLIC SERVICE CO CO,74444810,11,1,4931,PSR,8804,01,1966-12-15 00:00:00,1976-06-30 00:00:00
1001,25902,GENERAL STEEL INDUSTRIES INC,37085610,10,1,3229,GSI,5088,01,1965-01-07 00:00:00,1973-05-30 00:00:00
1964,68049,RYANS FAMILY STEAK HOUSES INC,78351910,11,3,5812,RYAN,9298,01,1989-09-14 00:00:00,1996-12-31 00:00:00
1984,59440,SAFECO CORP,78642910,11,1,6331,SAF,9351,01,1976-07-01 00:00:00,2008-09-22 00:00:00


In [25]:
sp500_const_list.describe()

,permno,shrcd,exchcd,hsiccd,gvkey
count,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000
mean,45047.839664,11.275090,1.316273,4466.340664,24298.312275
std,27185.642550,3.373588,0.939112,1899.850370,44243.266082
min,10006.000000,10.000000,-2.000000,0.000000,1010.000000
25%,20853.000000,11.000000,1.000000,3011.000000,4699.000000
50%,40221.000000,11.000000,1.000000,4011.000000,8325.000000
75%,71175.000000,11.000000,1.000000,6021.000000,13714.000000
max,93436.000000,72.000000,31.000000,9999.000000,316056.000000


## sp500_op_ret
Below are summary statistics (refer to page 15)

### general statistics

In [ ]:
sp500_op_ret.head(5)
# sp500_op_ret.sample(5)

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
0,5015,1996-01-31 00:00:00,1996-07-20 00:00:00,C,20.0,4.625,5.000,15,25,0.380062,0.793831,0.044693,4.456725,-2.603545,10980421,1,0,171,24.142672,24.000,24.0000,0.052253,4.8125,-0.025367
1,5048,1996-01-31 00:00:00,1996-08-17 00:00:00,P,40.0,3.000,3.375,5,37,0.245716,-0.507195,0.058756,11.186040,-1.508168,11613022,1,0,199,39.344930,39.000,39.0000,0.051780,3.1875,0.002347
2,5049,1996-01-31 00:00:00,1996-05-18 00:00:00,C,65.0,5.875,6.375,4,420,0.167298,0.833206,0.039200,9.020193,-5.515290,11618802,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721
3,5049,1996-01-31 00:00:00,1996-08-17 00:00:00,C,70.0,3.250,3.625,13,3806,0.149214,0.559395,0.050652,19.823520,-4.653782,11770703,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186
4,5061,1996-01-31 00:00:00,1996-06-22 00:00:00,C,30.0,3.375,3.750,5,254,0.324523,0.657941,0.056807,7.146084,-3.929316,10914516,1,0,143,32.001209,31.750,31.7500,0.052726,3.5625,-0.006649


In [97]:
sp500_op_ret[
    (sp500_op_ret["secid"] == 5049) & (sp500_op_ret["date"] == "1996-01-31 00:00:00")
]

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
2,5049,1996-01-31 00:00:00,1996-05-18 00:00:00,C,65.0,5.875,6.375,4,420,0.167298,0.833206,0.039200,9.020193,-5.515290,11618802,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721
3,5049,1996-01-31 00:00:00,1996-08-17 00:00:00,C,70.0,3.250,3.625,13,3806,0.149214,0.559395,0.050652,19.823520,-4.653782,11770703,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186
2065,5049,1996-01-31 00:00:00,1997-01-18 00:00:00,C,75.0,2.250,2.625,3,189,0.147161,0.371697,0.036495,25.504480,-3.143041,10637037,1,0,353,71.160224,70.375,35.1875,0.049987,2.4375,-0.000829
2066,5049,1996-01-31 00:00:00,1996-03-16 00:00:00,P,70.0,1.375,1.625,27,27,0.160386,-0.484632,0.107657,9.544870,-4.919761,11022992,1,0,45,70.183981,70.375,35.1875,0.054401,1.5000,-0.026413
2067,5049,1996-01-31 00:00:00,1996-05-18 00:00:00,P,70.0,2.000,2.375,1,67,0.148886,-0.473366,0.069962,14.875140,-1.948550,11185240,1,0,108,70.168661,70.375,35.1875,0.053274,2.1875,-0.009183
2068,5049,1996-01-31 00:00:00,1996-03-16 00:00:00,C,70.0,1.375,1.625,10,31,0.145853,0.526418,0.111908,9.628968,-7.829927,11680701,1,0,45,70.183981,70.375,35.1875,0.054401,1.5000,-0.029019


In [27]:
sp500_op_ret.describe()

,secid,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
count,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825393e+06
mean,1.118091e+05,1.583552e+02,1.048743e+01,1.099611e+01,8.288530e+01,1.758055e+03,3.903697e-01,1.159395e-01,3.800090e-02,2.467463e+01,-1.687396e+01,8.287611e+07,1.053916e+00,2.337976e-03,1.257383e+02,1.615546e+02,1.612870e+02,2.160093e+02,1.801193e-02,1.074177e+01,7.061732e-02
std,1.956878e+04,3.534783e+02,3.874286e+01,3.929525e+01,5.716760e+02,5.857573e+03,2.047089e-01,4.807320e-01,4.315247e-02,6.098045e+01,4.313121e+01,4.649504e+07,5.263510e-01,5.339109e-02,8.291730e+01,3.528938e+02,3.521467e+02,4.140947e+03,1.915548e-02,3.901709e+01,4.627202e+01
min,5.015000e+03,5.000000e-01,1.000000e-02,7.000000e-02,1.000000e+00,1.000000e+00,1.114700e-02,-9.996790e-01,0.000000e+00,4.900000e-04,-1.061745e+03,1.000001e+07,1.000000e+00,0.000000e+00,2.800000e+01,5.000000e-05,1.500000e-01,1.171875e-01,8.914000e-04,4.000000e-02,-1.601024e+04
25%,1.034340e+05,3.700000e+01,1.150000e+00,1.350000e+00,3.000000e+00,6.800000e+01,2.594700e-01,-2.718040e-01,1.047900e-02,5.147422e+00,-1.307261e+01,3.264821e+07,1.000000e+00,0.000000e+00,5.000000e+01,3.757191e+01,3.754000e+01,3.197000e+01,2.672931e-03,1.250000e+00,-3.809121e-02
50%,1.073980e+05,6.500000e+01,2.900000e+00,3.200000e+00,1.100000e+01,3.160000e+02,3.396460e-01,1.604910e-01,2.637400e-02,9.966705e+00,-6.169045e+00,1.033082e+08,1.000000e+00,0.000000e+00,1.090000e+02,6.498800e+01,6.486000e+01,5.673000e+01,1.074308e-02,3.050000e+00,-1.026760e-02
75%,1.116520e+05,1.290000e+02,7.400000e+00,7.900000e+00,4.000000e+01,1.264000e+03,4.580080e-01,5.068830e-01,5.078000e-02,2.019212e+01,-3.281715e+00,1.240800e+08,1.000000e+00,0.000000e+00,1.730000e+02,1.312548e+02,1.311500e+02,1.174850e+02,2.571517e-02,7.650000e+00,1.373601e-02
max,2.149050e+05,5.500000e+03,2.692950e+03,2.707550e+03,2.561690e+05,7.795400e+05,2.998749e+00,1.000030e+00,2.643942e+00,1.354885e+03,1.771834e+01,1.439230e+08,5.000000e+01,2.000000e+00,3.590000e+02,3.516114e+03,3.507070e+03,4.390544e+05,7.199510e-02,2.700250e+03,7.425289e+04


### number of put/call

In [33]:
num_call = sum(sp500_op_ret["cp_flag"] == "C")
num_put =  sum(sp500_op_ret["cp_flag"] == "P")
print(f"number of calls: {num_call}, comprising {round(num_call/(num_call + num_put) * 100)}% options")

number of calls: 2179336, comprising 57% options


### Moneyness

In [34]:
moneyness = sp500_op_ret["strike_price"] / sp500_op_ret["spotprice"]

In [60]:
sp500_op_ret["moneyness"] = moneyness

In [72]:
itm = sp500_op_ret[
    (
        (sp500_op_ret["moneyness"] > 1.1) & (sp500_op_ret["cp_flag"] == "P")
    )
    | 
    (
        (sp500_op_ret["moneyness"] < 0.9) & (sp500_op_ret["cp_flag"] == "C")
    )
]

In [73]:
otm = sp500_op_ret[
    (
        (sp500_op_ret["moneyness"] < 0.9) & (sp500_op_ret["cp_flag"] == "P")
    )
    | 
    (
        (sp500_op_ret["moneyness"] > 1.1) & (sp500_op_ret["cp_flag"] == "C")
    )
]

In [74]:
atm = sp500_op_ret[
        (sp500_op_ret["moneyness"] >= 0.9) & (sp500_op_ret["moneyness"] <= 1.1)
]

In [84]:
ioatm = [itm, otm, atm]
[round(i.shape[0] / sp500_op_ret.shape[0], 2) for i in ioatm]

[0.14, 0.33, 0.52]

In [80]:
sp500_op_ret.shape

(3825531, 25)

### average IV

In [15]:
sp500_op_ret["impl_volatility"].mean()

0.3903697014422311

### days to maturity

In [49]:
days2maturity = (    
    sp500_op_ret["exdate"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  - sp500_op_ret["date"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
)

In [45]:
days2maturity.describe()

count                        3825531
mean     125 days 17:43:05.652344728
std       82 days 22:00:54.621799602
min                 28 days 00:00:00
25%                 50 days 00:00:00
50%                109 days 00:00:00
75%                173 days 00:00:00
max                359 days 00:00:00
dtype: object

In [46]:
days2maturity[0]

Timedelta('171 days 00:00:00')

In [52]:
datetime.timedelta(900) - days2maturity[0]

Timedelta('729 days 00:00:00')

In [55]:
short_term_options = sum(days2maturity <= datetime.timedelta(90))
long_term_options = sum(days2maturity > datetime.timedelta(90))

In [56]:
short_term_options, long_term_options

(1764052, 2061479)

In [59]:
f"number of short_term_options {short_term_options} and its proportion {round(short_term_options / (short_term_options + long_term_options) * 100)}%"

'number of short_term_options 1764052 and its proportion 46%'

In [18]:
days2maturity.median()

Timedelta('109 days 00:00:00')

## Prediction

### just use very basic features --- almost no predictive power

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [43]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [54]:
from sklearn import linear_model

In [45]:
subsample = sp500_op_ret

In [46]:
subsample = subsample.dropna()

In [82]:
X = subsample[
    ["impl_volatility", "delta", "gamma", "vega", "theta"]
]

y = subsample['option_ret']

X_train, X_test, y_train, y_test = train_test_split(
    X, y
)

# regressor = LinearRegression()
regressor = linear_model.Lasso(alpha=0.1)
regressor.fit(X_train, y_train)

Lasso(alpha=0.1)

In [83]:
y_pred = regressor.predict(X_test)
mean_squared_error(y_test, y_pred)

1486.3248747034686

In [84]:
r2_score(y_test, y_pred)

-2.0995210787688734e-06

In [85]:
sp500_op_ret.shape

(3825531, 24)